In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import networkx as nx
from IPython.display import display, clear_output
from scipy import signal
from scipy import fftpack
from numpy.fft import fft, fftfreq
import os
import pandas as pd

# import custom module
import SIRS

In [ ]:
# needed only if you made some changes at SIRS.py and want to import the updated version of the module
from importlib import reload
reload(SIRS)

# LOOKING FOR CRITICAL VALUES OF THE DESEASE PARAMETERS

These parameters correspond to those of seasonal flu, in particular:

$$R_0 \in [0.9, 2.1]$$
recovery time $\in$ [5,7] so
$$\rightarrow \mu \in [0.143,0.2]$$
probability of contagion GIVEN contact 
$$\beta = R_0 \mu \in [0.128, 0.42]$$


If we raughly consider averages this value of $\beta$, in order to be used in our framework including topology, must be divided by the mean degree of the network $<k>$, so that $\widetilde{\beta} = \frac{\beta}{<k>}$ and the probability that a susceptible node gets infected can be computed as  
$P(I \rightarrow S) = \widetilde{\beta} \cdot [number\_of\_susceptible\_neightbours]$

In [ ]:
#mobility and topology parameters
N = 1000
I_sf_init = 5
I_er_init = 5
p_mob = 0
mean_degree = 4
eps = 0.1 

In [ ]:
#epidemics values
beta = 0.3/mean_degree #mean degree of generated networks is usually a little smaller
mu = 0.15
gamma = 0.016

R_0 = beta/mu
    
print("Basic Reproductive Ratio (taking into account topology):", R_0)
print("\u03B2 (taking into account topology):", beta)


infection_params = dict(beta=beta, mu=mu, gamma=gamma)


directory_name = str(beta)+"_"+str(mu)+"_"+str(gamma) 
os.system("mkdir "+directory_name)

This $\gamma$ value corresponds to an average time for going from R to S of 60 "days".

In [ ]:
n_runs = 30
n_iter = 1000

save_data = False
every_graph = 1

S_tot_er=0*(n_iter+1)
I_tot_er=0*(n_iter+1)
R_tot_er=0*(n_iter+1)
S_tot_sf=0*(n_iter+1)
I_tot_sf=0*(n_iter+1)
R_tot_sf=0*(n_iter+1)


for run in range(n_runs):   
    
    # prepare systems
    state_sf_init, state_er_init, variables_net_sf, variables_net_er = SIRS.prepare_two_sys(N, I_sf_init, I_er_init, 
                                                                                  p_mob, mean_degree)
    
    #saving initial states
    save = directory_name + "/state_sf_init_"+str(run)+".txt"
    np.savetxt(save, state_sf_init)

    save = directory_name + "/state_er_init_"+str(run)+".txt"
    np.savetxt(save, state_er_init)

    #saving initial variables: we just save the IDs of the travelling nodes and the initial network 
    #(as adjacency matrix)
    save = directory_name + "/A_sf_"+str(run)+".txt"
    np.savetxt(save, variables_net_sf["A_sf"])
    save = directory_name + "/A_er_"+str(run)+".txt"
    np.savetxt(save, variables_net_er["A_er"])


    save = directory_name + "/travellers_sf_"+str(run)+".txt"
    np.savetxt(save, variables_net_sf["travellers_sf"])
    save = directory_name + "/travellers_er_"+str(run)+".txt"
    np.savetxt(save, variables_net_er["travellers_er"])


    state_sf = np.copy(state_sf_init)
    state_er = np.copy(state_er_init)

    import time 
    start = time.time()
    S_sf = [N - I_sf_init]
    I_sf = [I_sf_init]
    R_sf = [0]
    S_er = [N - I_er_init]
    I_er = [I_er_init]
    R_er = [0]
    t_vec = []
    for i in range(n_iter):
        state_sf, state_er = SIRS.two_sys_full_SIRS_step(state_sf, state_er, **variables_net_sf, 
                                                **variables_net_er, **infection_params)
        S_sf.append(state_sf[:,0].sum())
        I_sf.append(state_sf[:,1].sum())
        R_sf.append(state_sf[:,2].sum())
        S_er.append(state_er[:,0].sum())
        I_er.append(state_er[:,1].sum())
        R_er.append(state_er[:,2].sum())
        t_vec.append(time.time()-start)

    tot_time = time.time()-start
    print("Total time elapsed: %.2f s"%tot_time)
    print("Time per iteration: %.4f s"%(tot_time/n_iter))

    S_sf = np.array(S_sf)
    I_sf = np.array(I_sf)
    R_sf = np.array(R_sf)
    S_er = np.array(S_er)
    I_er = np.array(I_er)
    R_er = np.array(R_er)
    t_vec = np.array(t_vec)
    
    if seva_data == True:
    
        #save timeseries of SIR
        SIR_sf = np.vstack((S_sf, I_sf, R_sf)).T
        save = directory_name+"/SIR_sf_"+str(run)+".txt"
        np.savetxt(save, SIR_sf)
        SIR_er = np.vstack((S_er, I_er, R_er)).T
        save = directory_name+"/SIR_er_"+str(run)+".txt"
        np.savetxt(save, SIR_er)
    
    S_tot_sf+=S_sf
    I_tot_sf+=I_sf
    R_tot_sf+=R_sf
    S_tot_er+=S_er
    I_tot_er+=I_er
    R_tot_er+=R_er
    
    if (run%every_graph==0): #plots every every_graph graphs

        indexes = np.arange(n_iter+1)
        fig, ax = plt.subplots(1,2, figsize=(15,8))
        ax[0].plot(indexes, S_sf, label = r'$S_{sf}$')
        ax[0].plot(indexes, I_sf, label = r'$I_{sf}$')
        ax[0].plot(indexes, R_sf, label = r'$R_{sf}$')
        ax[0].plot(indexes, S_sf+I_sf+R_sf, label = 'tot')
        ax[0].set_xlabel('Number of iterations', fontsize = 16)
        ax[0].set_ylabel('Number of individuals', fontsize = 16)
        ax[0].set_title("Scale-free network", fontsize = 16)
        ax[0].legend(fontsize=13, loc='upper right')

        ax[1].plot(indexes, S_er, label = r'$S_{er}$')
        ax[1].plot(indexes, I_er, label = r'$I_{er}$')
        ax[1].plot(indexes, R_er, label = r'$R_{er}$')
        ax[1].plot(indexes, S_er+I_er+R_er, label = 'tot')
        ax[1].set_title("Erdosh-Renyi network", fontsize = 16)
        ax[1].set_xlabel('Number of iterations', fontsize = 16)
        ax[1].set_ylabel('Number of individuals', fontsize = 16)
        ax[1].legend(fontsize=13, loc='upper right')

        plt.suptitle(r"$\beta= {:.3f}; \mu= {:.3f}; \gamma= {:.3f}$".format(beta, mu, gamma), fontsize=20)

        save=directory_name+"/run_"+str(run)+".png"
        fig.savefig(save)
        plt.close()
    

fig, ax = plt.subplots(1,2, figsize=(15,8))
ax[0].plot(indexes, S_tot_sf/n_runs, label = r'$S_{sf}$')
ax[0].plot(indexes, I_tot_sf/n_runs, label = r'$I_{sf}$')
ax[0].plot(indexes, R_tot_sf/n_runs, label = r'$R_{sf}$')
ax[0].plot(indexes, (S_tot_sf+I_tot_sf+R_tot_sf)/n_runs, label = 'tot')
ax[0].set_xlabel('Number of iterations', fontsize = 16)
ax[0].set_ylabel('Number of individuals', fontsize = 16)
ax[0].set_title("Scale-free network", fontsize = 16)
ax[0].legend(fontsize=13, loc='upper right')

ax[1].plot(indexes, S_tot_er/n_runs, label = r'$S_{er}$')
ax[1].plot(indexes, I_tot_er/n_runs, label = r'$I_{er}$')
ax[1].plot(indexes, R_tot_er/n_runs, label = r'$R_{er}$')
ax[1].plot(indexes, (S_tot_er+I_tot_er+R_tot_er)/n_runs, label = 'tot')
ax[1].set_title("Erdosh-Renyi network", fontsize = 16)
ax[1].set_xlabel('Number of iterations', fontsize = 16)
ax[1].set_ylabel('Number of individuals', fontsize = 16)
ax[1].legend(fontsize=13, loc='upper right')

plt.suptitle(r"Mean on {} runs, $\beta= {:.3f}; \mu= {:.3f}; \gamma= {:.3f}$".format(n_runs, beta, mu, gamma), fontsize=20)

save = directory_name+"/total.png"
fig.savefig(save)
plt.close()